In [1]:
import argparse
import numpy as np
import torch.nn as nn
import torch.utils.data
from torchsummary import summary
from utils.dataset import load_mat_hsi, sample_gt, HSIDataset
from utils.utils import split_info_print, metrics, show_results
from utils.scheduler import load_scheduler
from models.get_model import get_model
from utils.utils import Draw
import time
from sklearn.utils import shuffle
from tqdm import tqdm
import torch.backends.cudnn as cudnn

import os
from utils.utils import grouper, sliding_window, count_sliding_window

In [2]:
parser = argparse.ArgumentParser(description="run patch-based HSI classification")
parser.add_argument("--model", type=str, default='gscvit_HL_INT') # model name
parser.add_argument("--dataset_name", type=str, default="HongHu") # dataset name
parser.add_argument("--dataset_dir", type=str, default="../data") # dataset dir
parser.add_argument("--device", type=str, default="0")
parser.add_argument("--patch_size", type=int, default=8) # patch_size
parser.add_argument("--num_run", type=int, default=5)
parser.add_argument("--epoch", type=int, default=200)
parser.add_argument("--bs", type=int, default=128)  # bs = batch size
parser.add_argument("--ratio", type=float, default=0.02) # ratio of training + validation sample

opts,_ = parser.parse_known_args()

device = torch.device("cuda:{}".format(opts.device))

# print parameters
print("experiments will run on GPU device {}".format(opts.device))
print("model = {}".format(opts.model))
print("dataset = {}".format(opts.dataset_name))
print("dataset folder = {}".format(opts.dataset_dir))
print("patch size = {}".format(opts.patch_size))
print("batch size = {}".format(opts.bs))
print("total epoch = {}".format(opts.epoch))
print("{} for training, {} for validation and {} testing".format(opts.ratio / 2, opts.ratio / 2, 1 - opts.ratio))

# load data
image, gt, labels = load_mat_hsi(opts.dataset_name, opts.dataset_dir)

num_classes = len(labels)

num_bands = image.shape[-1]

# random seeds
seeds = [202401, 202402, 202403, 202404, 202405, 202406, 202407, 202408, 202409, 202410]

# empty list to storing results
results = []


experiments will run on GPU device 0
model = gscvit_HL_INT
dataset = HongHu
dataset folder = ../data
patch size = 8
batch size = 128
total epoch = 200
0.01 for training, 0.01 for validation and 0.98 testing


In [3]:
def setup_dataloaders(image, train_gt, val_gt, opts, seed):
    """
    设置数据加载器
    """
    # 创建数据集
    train_set = HSIDataset(image, train_gt, patch_size=opts.patch_size, data_aug=True)
    
    val_set = HSIDataset(image, val_gt, patch_size=opts.patch_size, data_aug=True)

    # 创建数据加载器
    generator = torch.Generator()
    generator.manual_seed(seed)

    train_loader = torch.utils.data.DataLoader(
        dataset=train_set,
        batch_size=opts.bs,
        shuffle=True,
        drop_last=False,
        generator=generator,
        worker_init_fn=lambda worker_id: np.random.seed(seed + worker_id)
    )

    val_loader = torch.utils.data.DataLoader(
        dataset=val_set,
        batch_size=opts.bs,
        shuffle=False,
        drop_last=False
    )

    return train_loader, val_loader

In [4]:
def setup_seed(seed):
    # PyTorch CPU随机种子
    torch.manual_seed(seed)
    # 所有GPU的随机种子
    torch.cuda.manual_seed_all(seed)
    torch.cuda.manual_seed(seed)
    # Python哈希种子（影响字典等数据结构的行为）
    os.environ['PYTHONHASHSEED'] = str(seed)
    # NumPy随机种子
    np.random.seed(seed)
    # Python内置随机种子
    random.seed(seed)
    # 启用确定性算法（降低性能但保证可重复）
    torch.backends.cudnn.deterministic = True
    # 关闭自动优化（固定卷积算法选择）
    torch.backends.cudnn.benchmark = False
    # 设置线程数，确保一致性
    torch.set_num_threads(1)

In [5]:
def train(network, optimizer, criterion, train_loader, val_loader, epoch, saving_path, device, scheduler=None):

    best_acc = -0.1
    losses = []

    for e in tqdm(range(1, epoch+1), desc=""):
        network.train()
        for batch_idx, (images, targets) in enumerate(train_loader):
            images, targets = images.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = network(images)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            losses.append(loss.item())
        if e % 10 == 0 or e == 1:
            mean_losses = np.mean(losses)
            train_info = "train at epoch {}/{}, loss={:.6f}"
            train_info = train_info.format(e, epoch,  mean_losses)
            tqdm.write(train_info)
            losses = []
        else:
            losses = []

        val_acc = validation(network, val_loader, device)

        if scheduler is not None:
            scheduler.step()

        is_best = val_acc >= best_acc
        best_acc = max(val_acc, best_acc)
        save_checkpoint(network, is_best, saving_path, epoch=e, acc=best_acc)


def validation(network, val_loader, device):
    num_correct = 0.
    total_num = 0.
    network.eval()
    for batch_idx, (images, targets) in enumerate(val_loader):
        images, targets = images.to(device), targets.to(device)
        outputs = network(images)
        _, outputs = torch.max(outputs, dim=1) 
        for output, target in zip(outputs, targets):
            num_correct = num_correct + (output.item() == target.item())
            total_num = total_num + 1
    overall_acc = num_correct / total_num
    return overall_acc


def test(network, model_dir, image, patch_size, n_classes, device):
    #network.load_state_dict(torch.load(model_dir + "/model_best.pth"))
    network.eval()

    patch_size = patch_size
    batch_size = 64
    window_size = (patch_size, patch_size)
    image_w, image_h = image.shape[:2]
    pad_size = patch_size // 2

    # pad the image
    image = np.pad(image, ((pad_size, pad_size), (pad_size, pad_size), (0, 0)), mode='reflect')

    probs = np.zeros(image.shape[:2] + (n_classes, ))

    iterations = count_sliding_window(image, window_size=window_size) // batch_size
    for batch in tqdm(grouper(batch_size, sliding_window(image, window_size=window_size)),
                      total=iterations,
                      desc="inference on the HSI"):
        with torch.no_grad():
            data = [b[0] for b in batch]
            data = np.copy(data)
            data = data.transpose((0, 3, 1, 2))
            data = torch.from_numpy(data)
            data = data.unsqueeze(1)

            indices = [b[1:] for b in batch]
            data = data.to(device)
            output = network(data)
            if isinstance(output, tuple):
                output = output[0]
            output = output.to('cpu').numpy()

            for (x, y, w, h), out in zip(indices, output):
                probs[x + w // 2, y + h // 2] += out
    return probs[pad_size:image_w + pad_size, pad_size:image_h + pad_size, :]


def save_checkpoint(network, is_best, saving_path, **kwargs):
    if not os.path.isdir(saving_path):
        os.makedirs(saving_path, exist_ok=True)

    if is_best:
        tqdm.write("epoch = {epoch}: best OA = {acc:.4f}".format(**kwargs))
        torch.save(network.state_dict(), os.path.join(saving_path, 'model_best.pth'))
    else:  # save the ckpt for each 10 epoch
        if kwargs['epoch'] % 10 == 0:
            torch.save(network.state_dict(), os.path.join(saving_path, 'model.pth'))



In [6]:
time_all = []
for run in range(opts.num_run):
    np.random.seed(seeds[run])
    torch.manual_seed(seeds[run])
    torch.cuda.manual_seed(seeds[run])
    cudnn.deterministic = True
    cudnn.benchmark = False
    print("running an experiment with the {} model".format(opts.model))
    print("run {} / {}".format(run + 1, opts.num_run))

    # get train_gt, val_gt and test_gt
    train_gt, val_gt, test_gt = sample_gt(
            gt, 
            train_samples_per_class=30,  # 每类30个训练样本
            val_samples_per_class=10,    # 每类10个验证样本
            seed=seeds[run]
        )

    # 设置数据加载器
    train_loader, val_loader = setup_dataloaders(
        image=image,
        train_gt=train_gt,
        val_gt=val_gt,
        opts=opts,
        seed=seeds[run]
    )

    # load model and loss
    model = get_model(opts.model, opts.dataset_name, opts.patch_size)
    model = model.to(device)
    
    model.set_qconfig()
    torch.ao.quantization.prepare_qat(model, inplace=True)
    if run == 0:
        split_info_print(train_gt, val_gt, test_gt, labels)
        print("network information:")
        with torch.no_grad():
            input_shape = (1, num_bands, opts.patch_size, opts.patch_size)
            summary(model, input_shape)

    model = model.to(device)
    # print(model)
    optimizer, scheduler = load_scheduler(opts.model, model)

    criterion = nn.CrossEntropyLoss()

    # where to save checkpoint model
    model_dir = "./checkpoints/" + opts.model + '/' + opts.dataset_name + '/' + str(run)

    try:
        train(model, optimizer, criterion, train_loader, val_loader, opts.epoch, model_dir, device, scheduler)
    except KeyboardInterrupt:
        print('"ctrl+c" is pused, the training is over')
    start = time.time()
    # test the model
    model.eval()
    torch.ao.quantization.convert(model, inplace=True)
    probabilities = test(model, model_dir, image, opts.patch_size, num_classes, device)

    prediction = np.argmax(probabilities, axis=-1)
    end = time.time()
    test_time = end - start
    time_all.append(test_time)
    # computing metrics
    run_results = metrics(prediction, test_gt, n_classes=num_classes)  # only for test set
    results.append(run_results)
    show_results(run_results, label_values=labels)

    # draw the classification map
    # Draw(model,image,gt,opts.patch_size,opts.dataset_name,opts.model,num_classes)
if opts.num_run > 1:
        show_results(results, label_values=labels, agregated=True)

running an experiment with the gscvit_HL_INT model
run 1 / 5
目标: 每类30个训练样本, 10个验证样本
------------------------------------------------------------
类别 0: 训练集 30样本, 验证集 10样本, 测试集 14001样本
类别 1: 训练集 30样本, 验证集 10样本, 测试集 3472样本
类别 2: 训练集 30样本, 验证集 10样本, 测试集 21781样本
类别 3: 训练集 30样本, 验证集 10样本, 测试集 163245样本
类别 4: 训练集 30样本, 验证集 10样本, 测试集 6178样本
类别 5: 训练集 30样本, 验证集 10样本, 测试集 44517样本
类别 6: 训练集 30样本, 验证集 10样本, 测试集 24063样本
类别 7: 训练集 30样本, 验证集 10样本, 测试集 4014样本
类别 8: 训练集 30样本, 验证集 10样本, 测试集 10779样本
类别 9: 训练集 30样本, 验证集 10样本, 测试集 12354样本
类别 10: 训练集 30样本, 验证集 10样本, 测试集 10975样本
类别 11: 训练集 30样本, 验证集 10样本, 测试集 8914样本
类别 12: 训练集 30样本, 验证集 10样本, 测试集 22467样本
类别 13: 训练集 30样本, 验证集 10样本, 测试集 7316样本
类别 14: 训练集 30样本, 验证集 10样本, 测试集 962样本
类别 15: 训练集 30样本, 验证集 10样本, 测试集 7222样本
类别 16: 训练集 30样本, 验证集 10样本, 测试集 2970样本
类别 17: 训练集 30样本, 验证集 10样本, 测试集 3177样本
类别 18: 训练集 30样本, 验证集 10样本, 测试集 8672样本
类别 19: 训练集 30样本, 验证集 10样本, 测试集 3446样本
类别 20: 训练集 30样本, 验证集 10样本, 测试集 1288样本
类别 21: 训练集 30样本, 验证集 10样本, 测试集 4000样本

验证采样结果:
-----------

  0%|▏                                         | 1/200 [00:00<02:09,  1.54it/s]

train at epoch 1/200, loss=2.660567
epoch = 1: best OA = 0.0455


  0%|▏                                         | 1/200 [00:01<02:09,  1.54it/s]

epoch = 2: best OA = 0.0500


...

epoch = 196: best OA = 0.8955


100%|████████████████████████████████████████| 200/200 [01:12<00:00,  2.75it/s]


train at epoch 200/200, loss=0.156779


inference on the HSI: 6999it [00:46, 149.71it/s]                               

Confusion matrix :
[[ 13750    136     24     47     33     11      0      0      0      0
       0      0      0      0      0      0      0      0      0      0
       0      0]
 [   205   3062    110      0      6      0      5     26      0      0
       0      0      4      0      0      0      9      0      1     44
       0      0]
 [   208    255  19601     10     36      0      2    235     25      0
       1     78     30     12      4      2      5      0      5     97
    1175      0]
 [   684      0    119 151208   7281      2     16     71      0      8
    2199     75     92      0     33      0      0      7     70     33
     498    849]
 [    94      9     46    324   5275      0      1     20      0      0
      52      7    156      0      0      0      0      0     12     13
     169      0]
 [    68      0     12     19     12  40582      0   1533      0      5
      73     37   1330    757      0      0      0      7     39      0
      43      0]
 [    14      7